<a href="https://colab.research.google.com/github/tayabr/stock-markets-analytics-zoomcamp/blob/main/Module_1_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# install main library YFinance
!pip install yfinance

In [3]:
# IMPORTS
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

In [4]:
end = date.today()
print(f'Year = {end.year}; month= {end.month}; day={end.day}')

start = date(year=end.year-70, month=end.month, day=end.day)
print(f'Period for indexes: {start} to {end} ')

Year = 2024; month= 4; day=22
Period for indexes: 1954-04-22 to 2024-04-22 


In [4]:
# Real Gross Domestic Product (GDPC1), Billions of Chained 2017 Dollars, Seasonally Adjusted Annual Rate
# https://fred.stlouisfed.org/series/GDPC1
gdpc1 = pdr.DataReader("GDPC1", "fred", start=start)

In [5]:
gdpc1['gdpc1_us_yoy'] = gdpc1.GDPC1/gdpc1.GDPC1.shift(4)-1
gdpc1['gdpc1_us_qoq'] = gdpc1.GDPC1/gdpc1.GDPC1.shift(4)-1
gdpc1.tail()

,GDPC1,gdpc1_us_yoy,gdpc1_us_qoq
DATE,,,
2022-10-01,21989.981,0.006517,0.006517
2023-01-01,22112.329,0.017179,0.017179
2023-04-01,22225.350,0.023825,0.023825
2023-07-01,22490.692,0.029269,0.029269
2023-10-01,22679.255,0.031345,0.031345


In [6]:
print(gdpc1.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 278 entries, 1954-07-01 to 2023-10-01
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   GDPC1         278 non-null    float64
 1   gdpc1_us_yoy  274 non-null    float64
 2   gdpc1_us_qoq  274 non-null    float64
dtypes: float64(3)
memory usage: 8.7 KB
None


In [7]:
# Filter the DataFrame to only include dates from 2023
gdpc1_2023 = gdpc1[gdpc1.index.year == 2023]

# Display the filtered DataFrame
print(gdpc1_2023)

                GDPC1  gdpc1_us_yoy  gdpc1_us_qoq
DATE                                             
2023-01-01  22112.329      0.017179      0.017179
2023-04-01  22225.350      0.023825      0.023825
2023-07-01  22490.692      0.029269      0.029269
2023-10-01  22679.255      0.031345      0.031345


In [16]:
# Calculate the average YoY growth rate for 2023
average_growth_rate = gdpc1_2023['gdpc1_us_yoy'].mean()

# Convert to percentage
average_growth_rate_percentage = average_growth_rate * 100

# Round percentage to the nearest tenth
percentage_rounded = round(average_growth_rate_percentage, 1)

print(f"Average YoY growth rate in 2023: {percentage_rounded:.1f}%")

Average YoY growth rate in 2023: 2.5%


In [9]:
# Market Yield on U.S. Treasury Securities at 2-Year Constant Maturity, Quoted on an Investment Basis (DGS2)
# https://fred.stlouisfed.org/series/DGS2
# Market Yield on U.S. Treasury Securities at 10-Year Constant Maturity, Quoted on an Investment Basis (DGS10)
# https://fred.stlouisfed.org/series/DGS10
dgs2 = pdr.DataReader("DGS2", "fred", start=start)

dgs10 = pdr.DataReader("DGS10", "fred", start=start)

In [10]:
print(dgs2.info(), dgs10.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12493 entries, 1976-06-01 to 2024-04-18
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DGS2    11968 non-null  float64
dtypes: float64(1)
memory usage: 195.2 KB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16253 entries, 1962-01-02 to 2024-04-18
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DGS10   15560 non-null  float64
dtypes: float64(1)
memory usage: 254.0 KB
None None


In [11]:
# Join on datetime index
result = dgs2.join(dgs10, how='inner')

print(result.head())


            DGS2  DGS10
DATE                   
1976-06-01  7.26   7.94
1976-06-02  7.23   7.94
1976-06-03  7.22   7.92
1976-06-04  7.12   7.89
1976-06-07  7.09   7.88


In [12]:
print(result.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 12493 entries, 1976-06-01 to 2024-04-18
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DGS2    11968 non-null  float64
 1   DGS10   11968 non-null  float64
dtypes: float64(2)
memory usage: 292.8 KB
None


In [15]:
# Calculate the difference between DGS10 and DGS2
result['DGS10_minus_DGS2'] = result['DGS10'] - result['DGS2']

# Filter dates from 1/1/2000 onward
filtered_result = result[result.index >= '2000-01-01']

# Calculate the minimum value of DGS10_minus_DGS2
min_value = filtered_result['DGS10_minus_DGS2'].min()

# Round min_value to the nearest tenth
min_value_rounded = round(min_value, 1)

print(f"The rounded minimum value of DGS10_minus_DGS2 is: {min_value_rounded:.1f}")

The rounded minimum value of DGS10_minus_DGS2 is: -1.1


In [17]:
# INDEXES from Yahoo Finance
# SP500 index (SNP - Free Realtime Quote in USD)
# WEB: https://finance.yahoo.com/quote/%5EGSPC/
sp500_data = yf.download(tickers="^GSPC", start="2019-04-09", end="2024-04-09")
ipc_data = yf.download(tickers="^MXX", start="2019-04-09", end="2024-04-09")

# Extract closing prices
sp500_close = sp500_data["Adj Close"]
ipc_close = ipc_data["Adj Close"]

# Calculate percentage growth
sp500_growth = (sp500_close.iloc[-1] / sp500_close.iloc[0] - 1) * 100
ipc_growth = (ipc_close.iloc[-1] / ipc_close.iloc[0] - 1) * 100

# Determine which index had higher growth
if sp500_growth > ipc_growth:
    higher_index = "S&P 500"
    higher_growth = round(sp500_growth)
else:
    higher_index = "IPC Mexico"
    higher_growth = round(ipc_growth)

print(f"The {higher_index} had a growth of approximately {higher_growth}% over the 5-year period.")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

The S&P 500 had a growth of approximately 81% over the 5-year period.


In [18]:
# Define the list of stock tickers
stock_tickers = ["2222.SR", "BRK-B", "AAPL", "MSFT", "GOOG", "JPM"]

# Fetch historical data for 2023
stock_data = yf.download(tickers=stock_tickers, start="2023-01-01", end="2023-12-31")

# Extract adjusted closing prices
adj_close_prices = stock_data["Adj Close"]

# Calculate maximum-minimum adjusted closing price for each stock
max_adj_close = adj_close_prices.max()
min_adj_close = adj_close_prices.min()

# Calculate the range ratio
range_ratio = (max_adj_close - min_adj_close) / max_adj_close

# Round the result to two decimal places
range_ratio_rounded = round(range_ratio, 2)

# Display the results
for ticker, ratio in zip(stock_tickers, range_ratio_rounded):
    print(f"{ticker}: Range Ratio = {ratio:.2f}")

# Determine the stock with the largest range ratio
max_range_stock = stock_tickers[range_ratio_rounded.argmax()]
print(f"\nThe stock with the largest range ratio is {max_range_stock}.")

[*********************100%%**********************]  6 of 6 completed


2222.SR: Range Ratio = 0.21
BRK-B: Range Ratio = 0.37
AAPL: Range Ratio = 0.21
MSFT: Range Ratio = 0.39
GOOG: Range Ratio = 0.28
JPM: Range Ratio = 0.42

The stock with the largest range ratio is JPM.


In [19]:
# Fetch dividend data for 2023
dividend_data = yf.download(tickers=stock_tickers, start="2023-01-01", end="2023-12-31", actions=True)

# Extract dividends and closing prices
dividends = dividend_data["Dividends"]
adj_close_prices = dividend_data["Adj Close"]

# Calculate total dividends paid in 2023
total_dividends = dividends.sum()

# Get the adjusted closing price on the last trading day of the year
last_day_adj_close = adj_close_prices.iloc[-1]

# Calculate dividend yield for each stock
dividend_yield = (total_dividends / last_day_adj_close) * 100

# Find the maximum value
max_dividend_yield = dividend_yield.max()

# Round to one decimal place
max_dividend_yield_rounded = round(max_dividend_yield, 1)

print(f"The maximum dividend yield in 2023 is approximately {max_dividend_yield_rounded}%.")

[*********************100%%**********************]  6 of 6 completed

The maximum dividend yield in 2023 is approximately 2.4%.


In [20]:
# Define the ticker for the S&P 500 (SPX)
sp500_ticker = "^GSPC"

# Fetch historical data for the S&P 500
sp500_data = yf.download(tickers=sp500_ticker, start="2019-01-01", end="2024-04-09")

# Calculate daily returns
sp500_data["Daily_Return"] = sp500_data["Adj Close"].pct_change()

# Calculate annualized volatility (standard deviation)
annual_volatility = sp500_data["Daily_Return"].std() * (252 ** 0.5)  # 252 trading days in a year

print(f"Annualized volatility of the S&P 500: {annual_volatility:.2%}")

[*********************100%%**********************]  1 of 1 completed

Annualized volatility of the S&P 500: 20.93%
